## Imports

In [1]:
#standard useful libraries
import pandas as pd
import numpy as np
import random

#model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#neural network dependencies
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

C:\Users\Amy Perez\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Data

In [2]:
#Loading data from csv
pipes_df = pd.read_csv("wMainData 20190214.csv")
pipes_df.head()

C:\Users\Amy Perez\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,2,3,4,5,6,7,8,10,11,12,59,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,WONUM,Distance,Match_addr,Address,Comments,ARC_Address,InitiatedDateTime,StartDateTime,CompletedDateTime,InitiatedYr,...,PRESSURE,MAX_HEAD,MIN_HEAD,AVE_HEAD,MAX_PRESS,MIN_PRESS,AVE_PRESS,DIFF_PRESS,MUSYM,Failure
0,725328,7.545442,"17 Cerro St SW, Atlanta, Georgia, 30314",Howell Mill Rd & 17th St NW,,HOWELL MILL RD & 17TH ST NW,1/9/2014,1/9/2014,1/9/2014,2014.0,...,102.1743,1199.7460,1178.2394,1191.5188,108.6975,99.3787,105.1327,9.3188,Ub,1
1,756318,2.714840,"Dekalb Ave NE & Elmira Pl NE, Atlanta, Georgia...",DeKalb Ave NE & Elmira Pl NE,2ND WATER MAIN BREAK AT DEKALB AVE NE AND ELMI...,DeKalb Ave NE & Elmira Pl NE,4/18/2014,4/20/2015,4/20/2015,2014.0,...,69.7252,1197.4366,1176.4850,1187.7832,74.7168,65.6384,70.5340,9.0784,Ub,1
2,1349957,8.354273,"Evans Dr SW & Astor Ave SW, Atlanta, Georgia, ...",Evans Dr SW & Astor Ave SW,Call taken by: DROBERTSON,Evans Dr SW & Astor Ave SW,11/2/2015,NaN,NaN,2015.0,...,81.9982,1196.2609,1148.9755,1177.9572,97.6555,77.1668,89.7245,20.4888,Ub,1
3,780444,14.600671,"Campbellton Fairburn Rd & Ridge Rd, Fairburn, ...",CAMPBELLTON FAIRBURN RD & RIDGE RD,PER JOHN SKINNER THIS IS A KNOCKED OFF HYDRANT...,CAMPBELLTON FAIRBURN RD & RIDGE RD,7/6/2014,7/6/2014,7/6/2014,2014.0,...,128.9117,1166.0968,1134.0135,1151.1066,136.5314,122.6298,130.0362,13.9017,AgB,1
4,774585,15.977450,"1 Hartsfield Center Pkwy, Atlanta, Georgia, 30354",1 HARTSFIELD CENTER PKWY SW,,1 HARTSFIELD CENTER PKWY SW,6/6/2014,6/6/2014,6/6/2014,2014.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,Ub,1


In [4]:
#get dummies for columns where applicable--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['ZONE'], prefix='P-Zone')], axis=1)
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['MATERIAL_12'], prefix='Material')], axis=1); 

#drop original columns--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df2 = pipes_df.drop(columns={ 'MATERIAL_12','ZONE'})

#Delete unneeded columns
#NOTE to Team: I didn’t see any data at all in the ‘DEMAND9’, ‘PATTERN9’, ‘DEMAND10’, ‘PATTERN10’ columns,so I deleted all of these columns
pipes_df3 = pipes_df2.drop(columns={"PATTERN1", "PATTERN2", "PATTERN3", "PATTERN4", "PATTERN5", "PATTERN6", "PATTERN7", 
                                    "PATTERN8", "PATTERN9", "PATTERN10", "DEMAND9", "DEMAND10",'WONUM','Distance',
                                    'Match_addr','Address','Comments','ARC_Address','StartDateTime',
                                    'CompletedDateTime','DESCRIPTION','SrvcRqstTypDesc','MATERIAL'})

In [5]:
#Add a column to be opposite of FAILURE column for use in neural network training
tempSuccessList = []
for row in np.arange(len(pipes_df3['MOID'])):
    value = 1 - pipes_df3.iloc[row]['Failure']
    tempSuccessList.append(value)
pipes_df3['SUCCESS'] = tempSuccessList

# Machine Learning

# PreProcessing

In [6]:
pipes_df3.head()
pipes_df3.columns.values.tolist()


['InitiatedDateTime',
 'InitiatedYr',
 'INSTALLDATE_12',
 'Year_Installed',
 'AGE',
 'DIAMETER_1',
 'PLATCARD_12_13',
 'DIAMETER_2',
 'Length',
 'DIAMETER_12_13',
 'ROUGHNESS',
 'MOID',
 'STREETNAME',
 'LINEYEAR',
 'INDEX_DIAM',
 'INDEX_YR',
 'LENGTH',
 'FLOW',
 'VELOCITY',
 'HEADLOSS',
 'HL1000',
 'MAX_FLOW',
 'MIN_FLOW',
 'AVE_FLOW',
 'MAX_VELOC',
 'MIN_VELOC',
 'AVE_VELOC',
 'MAX_HDLOSS',
 'MIN_HDLOSS',
 'AVE_HDLOSS',
 'ELEVATION',
 'DEMAND1',
 'DEMAND2',
 'DEMAND3',
 'DEMAND4',
 'DEMAND5',
 'DEMAND6',
 'DEMAND7',
 'DEMAND8',
 'RUN_ELEV',
 'DEMAND',
 'HEAD',
 'PRESSURE',
 'MAX_HEAD',
 'MIN_HEAD',
 'AVE_HEAD',
 'MAX_PRESS',
 'MIN_PRESS',
 'AVE_PRESS',
 'DIFF_PRESS',
 'MUSYM',
 'Failure',
 'P-Zone_Chatt 1020',
 'P-Zone_Hemp 1175',
 'P-Zone_North 1225',
 'P-Zone_WTP',
 'Material_ ',
 'Material_AC',
 'Material_CAS',
 'Material_COP',
 'Material_DIP',
 'Material_GP',
 'Material_PVC',
 'Material_RCP',
 'Material_SP',
 'Material_UNK',
 'Material_W.I.',
 'SUCCESS']

In [28]:
#Select which columns to train off of.
#A number of iterations of this were initially used when exploring the data
#In the end it was decided to only use average values for those which have averages

inputColumns =['DIAMETER_2','ROUGHNESS','AGE','LENGTH','FLOW','VELOCITY','HEADLOSS','HL1000','MAX_FLOW','MIN_FLOW',
               'AVE_FLOW','MAX_VELOC','MIN_VELOC','AVE_VELOC','MAX_HDLOSS','MIN_HDLOSS','AVE_HDLOSS','DEMAND','MAX_HEAD',
               'MIN_HEAD','AVE_HEAD','MAX_PRESS','MIN_PRESS','AVE_PRESS','DIFF_PRESS','P-Zone_Chatt 1020',
               'P-Zone_Hemp 1175','P-Zone_North 1225','P-Zone_WTP','Material_AC','Material_CAS','Material_COP',
               'Material_DIP','Material_GP','Material_PVC','Material_RCP','Material_SP','Material_UNK','Material_W.I.']

In [29]:
#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pipes_df3[inputColumns],pipes_df3['Failure'],stratify=pipes_df3['Failure'],test_size = 0.3, random_state=42)

#Make another split for y which uses the additional 'SUCCESS' column for use in neural network
#Note: Using random state the X_train and X_test would be identical and so need not be kept
_, _, y_train_2, y_test_2 = train_test_split(pipes_df3[inputColumns],pipes_df3[['Failure','SUCCESS']],stratify=pipes_df3['Failure'],test_size = 0.3, random_state=42)

In [30]:
#Note: If we were to train and test on the split as it currently is, it will just guess that everything is an unbroken pipe
#This is not desired behavior

#Split X_train into a list of those where the pipes are broken and those which arent
#Then, undersample those pipes which are unbroken to avoid bias
X_train_false = X_train[y_train==0].sample(n=sum(y_train), random_state=42)
X_train_true = X_train[y_train==1].sample(n=sum(y_train), random_state=42)

#Recombine the sampled X_train into one list and keep only those y_train values corresponding to selected indices
X_train_sampled = X_train_true.append(X_train_false)
y_train_sampled = y_train.loc[X_train_sampled.index]
y_train_sampled_2 = y_train_2.loc[X_train_sampled.index]

#Apply min-max scalers to X
#Note: applying min-max too early changed format of dataframe into a np.array and interfered with the sampling process
X_minmax = MinMaxScaler().fit(X_train)
X_train_minmax = X_minmax.transform(X_train_sampled)
X_test_minmax = X_minmax.transform(X_test)

## Logistic Regression

In [52]:
#Set a logistic regression as a model
lrmodel = LogisticRegression()
lrmodel.fit(X_train_minmax,y_train_sampled)
lrmodel.score(X_train_minmax,y_train_sampled)

0.7532356948228883

In [53]:
lrmodel.score(X_test_minmax,y_test)

0.7309427217793413

In [54]:
confusion_matrix(y_test,lrmodel.predict(X_test_minmax))

array([[23837,  8896],
       [  588,  1928]], dtype=int64)

## Neural Network

In [12]:
#Add a shallow neural network
nnmodel = Sequential()

#Add layers
nnmodel.add(Dense(100, activation='relu', input_dim=X_train_minmax.shape[1]))
nnmodel.add(Dense(2,activation = 'softmax'))

#Compile
nnmodel.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

#Fit the training model to the data
nnmodel.fit(
    X_train_minmax,
    y_train_sampled_2,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 0.6586 - acc: 0.6074
Epoch 2/100
 - 0s - loss: 0.6410 - acc: 0.6277
Epoch 3/100
 - 0s - loss: 0.6306 - acc: 0.6413
Epoch 4/100
 - 0s - loss: 0.6224 - acc: 0.6489
Epoch 5/100
 - 0s - loss: 0.6120 - acc: 0.6619
Epoch 6/100
 - 0s - loss: 0.6008 - acc: 0.6790
Epoch 7/100
 - 0s - loss: 0.5971 - acc: 0.6786
Epoch 8/100
 - 0s - loss: 0.5882 - acc: 0.6899
Epoch 9/100
 - 0s - loss: 0.5823 - acc: 0.6960
Epoch 10/100
 - 0s - loss: 0.5776 - acc: 0.7018
Epoch 11/100
 - 0s - loss: 0.5699 - acc: 0.7127
Epoch 12/100
 - 0s - loss: 0.5654 - acc: 0.7194
Epoch 13/100
 - 0s - loss: 0.5616 - acc: 0.7131
Epoch 14/100
 - 0s - loss: 0.5571 - acc: 0.7245
Epoch 15/100
 - 0s - loss: 0.5570 - acc: 0.7240
Epoch 16/100
 - 0s - loss: 0.5559 - acc: 0.7255
Epoch 17/100
 - 0s - loss: 0.5500 - acc: 0.7265
Epoch 18/100
 - 0s - loss: 0.5512 - acc: 0.7263
Epoch 19/100
 - 0s - loss: 0.5493 - acc: 0.7240
Epoch 20/100
 - 0s - loss: 0.5450 - acc: 0.7307
Epoch 21/100
 - 0s - loss: 0.5456 - acc: 0.7255
E

In [13]:
nnmodel_loss, nnmodel_accuracy = nnmodel.evaluate(X_test_minmax, y_test_2, verbose=2)
print(f"Loss: {nnmodel_loss}, Accuracy: {nnmodel_accuracy}")

Loss: 0.4486992735858192, Accuracy: 0.8023089740537968


## Random Forests

### Decide on number of estimators to use

In [34]:
#Load a Random Forest model
model = RandomForestClassifier()

#Set a parameter grid
#Only real parameters we needed to check was the number of estimators
param_grid = {'n_estimators': [20,50,100,200,300]}
grid = GridSearchCV(model, param_grid, verbose=3)

#Fit the grid
grid.fit(X_train_minmax,y_train_sampled)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=20 .................................................
[CV] ........ n_estimators=20, score=0.8455056179775281, total=   1.0s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV] ......... n_estimators=20, score=0.845426673479816, total=   1.0s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s


[CV] ........ n_estimators=20, score=0.8390393459376597, total=   1.4s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.8503575076608785, total=   2.8s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.8546244251405212, total=   2.2s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.8530914665304037, total=   2.6s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.8549540347293156, total=   5.4s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.8571793561573837, total=   5.5s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.8515585079202862, total=   4.5s
[CV] n_estimators=200 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [35]:
#Find the best score and best parameters
print(f'The best score was {grid.score(X_test_minmax,y_test)}\nThe best parameters were {grid.best_params_}')

The best score was 0.8357400209935033
The best parameters were {'n_estimators': 300}


In [36]:
#Take a look at classification report to confirm that the model is ideal
print(classification_report(y_test, grid.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       0.99      0.83      0.90     32733
          1       0.29      0.90      0.44      2516

avg / total       0.94      0.84      0.87     35249



In [37]:
#Take a look at the confusion matrix to confirm that there is no unintended behavior (e.g. only guessing unbroken)
confusion_matrix(y_test, grid.predict(X_test_minmax))

array([[27193,  5540],
       [  250,  2266]], dtype=int64)

# Performing deeper analysis with Random Forests

In [38]:
#Apply cross validation so as to have percentage predictions for all data points without bias towards training data
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
split = cv.split(pipes_df3[inputColumns],pipes_df3['Failure'])
rfmodel = RandomForestClassifier(n_estimators=300, random_state=42)

In [39]:
#Create new column for use in displaying results of analysis to be filled in later
#Final results will be stored in final_df
scores = []
final_df = pipes_df3
final_df['BurstChance'] = 0
confusion = []

In [40]:
#For each train-test split from our crossvalidation, run the same type of analysis as before
for train_idx, test_idx in cv.split(pipes_df3[inputColumns],pipes_df3['Failure']):
    Current_X_train = pipes_df3[inputColumns].loc[train_idx]
    Current_X_test = pipes_df3[inputColumns].loc[test_idx]
    Current_y_train = pipes_df3['Failure'].loc[train_idx]
    Current_y_test = pipes_df3['Failure'].loc[test_idx]
    
    
    #Undersample training data as before
    Current_X_train_false = Current_X_train[Current_y_train==0].sample(n=sum(Current_y_train))
    Current_X_train_true = Current_X_train[Current_y_train==1].sample(n=sum(Current_y_train))
    
    Current_X_train_sampled = Current_X_train_true.append(Current_X_train_false)
    Current_y_train_sampled = Current_y_train.loc[Current_X_train_sampled.index]
    
    Current_X_minmax = MinMaxScaler().fit(Current_X_train)
    Current_X_train_minmax = Current_X_minmax.transform(Current_X_train_sampled)
    Current_X_test_minmax = Current_X_minmax.transform(Current_X_test)

    
    rfmodel.fit(Current_X_train_minmax, Current_y_train_sampled)
    score = rfmodel.score(Current_X_test_minmax, Current_y_test)
    scores.append(score)
    currentconfusion = confusion_matrix(Current_y_test,rfmodel.predict(Current_X_test_minmax))
    print(currentconfusion)
    confusion.append([currentconfusion[0][0],currentconfusion[0][1],currentconfusion[1][0],currentconfusion[1][1]])
    
    #Store prediction probabilities into final_df
    final_df['BurstChance'].loc[test_idx] = [x[1] for x in list(rfmodel.predict_proba(Current_X_test_minmax))]


[[18342  3480]
 [  142  1536]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[18565  3256]
 [  172  1506]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[18632  3189]
 [  171  1507]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[18291  3530]
 [  163  1514]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[18441  3380]
 [  182  1495]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [41]:
#Look at combined confusion matrix
truepositive = confusion[0][0] + confusion[1][0]+confusion[2][0]+confusion[3][0]+confusion[4][0]
falsenegative = confusion[0][1] + confusion[1][1]+confusion[2][1]+confusion[3][1]+confusion[4][1]
falsepositive = confusion[0][2] + confusion[1][2]+confusion[2][2]+confusion[3][2]+confusion[4][2]
truenegative = confusion[0][3] + confusion[1][3]+confusion[2][3]+confusion[3][3]+confusion[4][3]


In [42]:
print(f'[{[truepositive, falsenegative]}\n{[falsepositive,truenegative]}]')

[[92271, 16835]
[830, 7558]]


In [43]:
#Look at final scores for each
scores

[0.8458723404255319,
 0.8541214519766799,
 0.8570151921358355,
 0.842837688313899,
 0.8484126308622011]

In [57]:
#Output top ten most likely pipes to burst which are currently unbroken according to our model
out = final_df[['BurstChance','MOID'] + inputColumns].loc[final_df['Failure']==0].sort_values('BurstChance', ascending=False)


,BurstChance,MOID,DIAMETER_2,Length,ROUGHNESS,AGE,LENGTH,FLOW,VELOCITY,HEADLOSS,...,Material_AC,Material_CAS,Material_COP,Material_DIP,Material_GP,Material_PVC,Material_RCP,Material_SP,Material_UNK,Material_W.I.
14811,1.0,P52661,2.0,194.184290,30,78,194.184290,1.1791,0.1204,0.1732,...,0,0,0,0,0,0,0,0,0,1
95981,1.0,P148456,6.0,9.058445,121,53,9.058445,0.0000,0.0000,0.0000,...,0,1,0,0,0,0,0,0,0,0
11502,1.0,P35890,2.0,496.622452,40,42,496.622452,0.6176,0.0631,0.0785,...,0,0,0,0,0,0,0,0,0,1
93480,1.0,P50595,2.0,424.786041,30,58,424.786041,0.7623,0.0779,0.1688,...,0,0,0,0,0,0,0,0,0,1
97912,1.0,P174490,6.0,7.081598,121,55,7.081598,0.0000,0.0000,0.0000,...,0,1,0,0,0,0,0,0,0,0


In [50]:
#Output most relevant parameters according to our recent model
print(sorted(zip(map(lambda x: round(x, 4), rfmodel.feature_importances_), inputColumns), reverse=True))

[(0.2334, 'AGE'), (0.0821, 'Material_DIP'), (0.0491, 'ROUGHNESS'), (0.0432, 'LENGTH'), (0.0408, 'Length'), (0.0364, 'Material_CAS'), (0.0336, 'MAX_HEAD'), (0.0326, 'DIAMETER_2'), (0.0322, 'MIN_HEAD'), (0.0319, 'AVE_HEAD'), (0.0305, 'DEMAND'), (0.0284, 'DIFF_PRESS'), (0.0247, 'MIN_PRESS'), (0.0241, 'MAX_PRESS'), (0.0237, 'AVE_PRESS'), (0.0209, 'AVE_FLOW'), (0.0208, 'MAX_FLOW'), (0.0207, 'FLOW'), (0.0198, 'MIN_FLOW'), (0.0191, 'MAX_VELOC'), (0.0187, 'MIN_VELOC'), (0.0186, 'AVE_VELOC'), (0.0182, 'VELOCITY'), (0.0173, 'AVE_HDLOSS'), (0.0167, 'MAX_HDLOSS'), (0.016, 'HEADLOSS'), (0.0159, 'MIN_HDLOSS'), (0.0115, 'HL1000'), (0.0049, 'Material_W.I.'), (0.0045, 'Material_UNK'), (0.0038, 'P-Zone_North 1225'), (0.0038, 'P-Zone_Hemp 1175'), (0.0013, 'P-Zone_Chatt 1020'), (0.0003, 'Material_SP'), (0.0002, 'Material_COP'), (0.0001, 'P-Zone_WTP'), (0.0001, 'Material_PVC'), (0.0001, 'Material_GP'), (0.0, 'Material_RCP'), (0.0, 'Material_AC')]


In [27]:
len(inputColumns)

57

In [51]:
print(classification_report(y_test,rfmodel.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       1.00      0.85      0.92     32733
          1       0.33      0.98      0.49      2516

avg / total       0.95      0.86      0.89     35249



In [59]:
out.to_csv('output.csv')
